# Tác vụ 1b: Tạo văn bản bằng câu lệnh bao gồm Ngữ cảnh

Trong sổ tay này, bạn sẽ học cách tạo email hồi đáp khách hàng không hài lòng với chất lượng của dịch vụ khách hàng mà họ nhận được từ kỹ sư hỗ trợ khách hàng. Bạn sẽ cung cấp ngữ cảnh bổ sung cho mô hình bằng cách đưa vào nội dung của email thực tế nhận được từ khách hàng không hài lòng.

PromptTemplates sẽ hỗ trợ bạn tăng thêm độ phức tạp để tận dụng khung LangChain cho một trường hợp sử dụng tương tự. PromptTemplates sẽ giúp bạn tạo các shell chung mà có thể được điền thông tin sau này và thu được đầu ra của mô hình dựa trên nhiều tình huống.

[LangChain](https://python.langchain.com/docs/get_started/introduction.html) là một khung phát triển ứng dụng do các mô hình ngôn ngữ hỗ trợ. Các khía cạnh chính của khung này giúp chúng tôi tăng cường các Mô hình ngôn ngữ lớn bằng cách xâu chuỗi nhiều thành phần với nhau để tạo ra những trường hợp sử dụng nâng cao.

Do có ngữ cảnh bổ sung trong câu lệnh nên nội dung được tạo ra trong sổ tay này có chất lượng và mức độ phù hợp cao hơn nhiều so với nội dung được tạo trước đó thông qua câu lệnh không cần mẫu (zero-shot). Câu lệnh dùng trong sổ tay này sẽ tạo ra một mẫu câu lệnh LangChain tùy chỉnh để thêm ngữ cảnh vào yêu cầu tạo văn bản.

#### Kịch bản
Bạn đóng vai Bob, Giám đốc Dịch vụ khách hàng tại AnyCompany. Một số khách hàng của bạn đang không hài lòng với dịch vụ khách hàng của công ty và phản hồi tiêu cực về dịch vụ do các kỹ sư hỗ trợ khách hàng cung cấp. Bây giờ, bạn muốn hồi đáp những khách hàng đó thật khiêm tốn để xin lỗi về chất lượng dịch vụ kém và lấy lại lòng tin của họ. Bạn cần sự trợ giúp của Mô hình ngôn ngữ lớn (LLM) để tạo ra một lượng lớn email phù hợp với con người và được cá nhân hóa dựa trên cảm xúc của khách hàng trong các email trao đổi trước đó.

Trong trường hợp này, bạn có thể tận dụng sức mạnh có trong PromptTemplates của LangChain để tạo ra một shell chung. Bạn sẽ dùng shell này để tạo email hồi đáp được cá nhân hóa dựa trên email trước của khách hàng. PromptTemplate sẽ kết hợp nội dung trong email gốc của khách hàng để LLM hiểu được ngữ cảnh và cảm xúc, sau đó tạo phản hồi phù hợp và tùy chỉnh.

## Tác vụ 1b.1: Thiết lập môi trường

Trong tác vụ này, bạn sẽ thiết lập môi trường của mình.

In [ ]:
#Create a service client by name using the default session.
import json
import os
import sys
import warnings

import boto3

warnings.filterwarnings('ignore')
module_path = ".."
sys.path.append(os.path.abspath(module_path))


bedrock_client = boto3.client('bedrock-runtime',region_name=os.environ.get("AWS_DEFAULT_REGION", None))

## Tác vụ 1b.2: Gọi Mô hình LLM Bedrock

Trong tác vụ này, bạn sẽ tạo một phiên bản của lớp Bedrock từ các LLM. Thao tác này cần có `model_id` là Tên tài nguyên Amazon (ARN) của mô hình có trong Amazon Bedrock.

(Không bắt buộc) Bạn có thể chuyển một ứng dụng boto3 đã tạo trước đó cũng như một số `model_kwargs` có thể chứa các tham số như `temperature`, `top_p`, `max_token_count` hoặc `stop_sequences` (bạn có thể khám phá thêm về các tham số trong bảng điều khiển Amazon Bedrock).

Tham khảo [tài liệu](https://docs.aws.amazon.com/bedrock/latest/userguide/model-ids-arns.html) để biết các mã mô hình tạo văn bản có trong Amazon Bedrock.

<i aria-hidden="true" class="fas fa-sticky-note" style="color:#563377"></i> **Lưu ý:** Các mô hình khác nhau hỗ trợ các `model_kwargs` khác nhau.

In [ ]:
# Model configuration
from langchain_aws import ChatBedrock
from langchain_core.output_parsers import StrOutputParser

model_id = "meta.llama3-8b-instruct-v1:0"
model_kwargs =  { 
        "max_gen_len": 512,
        "temperature": 0,
        "top_p": 1,
}

# LangChain class for chat
chat_model = ChatBedrock(
    client=bedrock_client,
    model_id=model_id,
    model_kwargs=model_kwargs,
)

## Tác vụ 1b.3: Tạo mẫu câu lệnh tùy chỉnh LangChain

Trong tác vụ này, bạn sẽ tạo mẫu cho câu lệnh mà bạn có thể chuyển các biến đầu vào khác nhau trong mỗi lần chạy. Điều này rất hữu ích khi bạn phải tạo nội dung có các biến đầu vào khác nhau mà bạn có thể tìm nạp từ cơ sở dữ liệu.

Trong tác vụ trước, chúng ta đã mã hóa cứng câu lệnh. Có thể bạn đã nhận được phản hồi tiêu cực tương tự từ nhiều khách hàng, nên giờ bạn muốn sử dụng email của từng khách hàng đó và gửi lời xin lỗi họ. Tuy nhiên, bạn cũng muốn phản hồi sẽ được cá nhân hóa một chút. Trong ô sau, bạn sẽ khám phá cách tạo `PromptTemplate` để đạt được mẫu hình này.

In [ ]:
# Create a prompt template that has multiple input variables
from langchain.prompts import PromptTemplate

multi_var_prompt = PromptTemplate(
    input_variables=["customerServiceManager", "customerName", "feedbackFromCustomer"], 
    template="""

Human: Create an apology email from the Service Manager {customerServiceManager} at AnyCompany to {customerName} in response to the following feedback that was received from the customer: 
<customer_feedback>
{feedbackFromCustomer}
</customer_feedback>

Assistant:"""
)

# Pass in values to the input variables
prompt = multi_var_prompt.format(customerServiceManager="Bob Smith", 
                                 customerName="John Doe", 
                                 feedbackFromCustomer="""Hello Bob,
     I am very disappointed with the recent experience I had when I called your customer support.
     I was expecting an immediate call back but it took three days for us to get a call back.
     The first suggestion to fix the problem was incorrect. Ultimately the problem was fixed after three days.
     We are very unhappy with the response provided and may consider taking our business elsewhere.
     """
     )


In [ ]:
# get number of tokens
num_tokens = chat_model.get_num_tokens(prompt)
print(f"Our prompt has {num_tokens} tokens")

<i aria-hidden="true" class="fas fa-sticky-note" style="color:#563377"></i> **Lưu ý:** Bạn có thể bỏ qua các cảnh báo một cách an toàn và chuyển sang ô tiếp theo.

In [ ]:
#invoke
response = chat_model.invoke(prompt)

In [ ]:
# Configure a Chain to parse output
chain = StrOutputParser()
formatted_response=chain.invoke(response)
print(formatted_response)

Bạn đã học được rằng việc gọi LLM không có ngữ cảnh sẽ không mang lại kết quả mong muốn. Khi bổ sung ngữ cảnh và sử dụng thêm mẫu câu lệnh để hạn chế đầu ra từ LLM, bạn đã có thể thu được đầu ra mong muốn.

### Tự thực hiện
- Thay đổi câu lệnh theo trường hợp sử dụng cụ thể của bạn và đánh giá đầu ra của các mô hình khác nhau.
- Thử nghiệm với độ dài mã token để nắm được độ trễ và khả năng phản hồi của dịch vụ.
- Áp dụng các nguyên tắc tạo câu lệnh khác nhau để có được đầu ra tốt hơn.

### Dọn dẹp

Bạn đã hoàn thành sổ tay này. Để chuyển sang phần tiếp theo của phòng thực hành, hãy làm như sau:

- Đóng tệp sổ tay này.
- Quay lại phiên thực hành và tiếp tục với **Tác vụ 2**.